In [3]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from string import digits
from collections import Counter
import csv
import re
import io
import inflection
import os

In [4]:
import pandas as pd

In [5]:
data=pd.read_csv('reviews_data.csv')

In [6]:
print(data.shape)

(5000, 24)


In [7]:
print(data.columns)

Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')


In [8]:
products_unique=data.iloc[:]['name'].unique()

In [9]:
product_details=data.groupby('name')['reviews.text'].apply(' '.join).reset_index()

In [10]:
product_count=data.groupby('name').count()

In [11]:
product_details.to_csv('product_grouped.csv',index=False)

In [12]:
#split sentences
caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [13]:
def decontracted(text):
    
    
    # specific
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)

    # general
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

In [14]:
def unwanted_char(text):
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text,flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'.[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text=text.translate(str.maketrans('','','1234567890'))
    return text

In [15]:
data=pd.read_csv('product_grouped.csv')

In [16]:
text_pro=data.iloc[0]['reviews.text']
print(text_pro)


I love this tablet. I purchased them for gift and am so glad I did! Looks pretty solid will update review as soon as gift is opened . for the price this is a great works well the for the price display is hd love it Great tablet for under $100. Does not have the highest specs, but gets the job done for lite tablet users. Purchased mainly to use travelling so haven't really had the opportunity to give it a fair road test at this point. It was advertised as capable of downloading movies and books so you can watch/read while travelling without burning through your data or being hit with data roaming fees or having to purchase wifi during flights. This is my second Kindle and I love it. It's lightweight, simple, and fast. It's convenient for trips and fits in my bag without weighing it down. It's a good starter tablet, to bad it runs on the Amazon OS. I bought these as a starter online browser. It is a very portable pad . Great price. The kindle fire hd is a sweet tablet for reading and app

In [17]:
f= open("big.txt","w+")
f.write(text_pro)
f.close()

In [18]:
#spell corrector
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def spelling_corrector(text): 
    text=text.split()   
    text=[correction(w) for w in text]
    text=" ".join(text)
    return text

In [19]:
#funtion that calls all the preprocessing function
def clean_text(text):
    
    #convert to lower case
    text=text.lower()
    
    #replace contractions with their expanded  form
    text=decontracted(text)
    
    #remove unwanted characters
    text=unwanted_char(text)
    
    #remove stopwords
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    
    #singularize
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [inflection.singularize(w) for w in text]
    text = " ".join(text)
    
    #spell corrector 
    text=spelling_corrector(text)
    
    #lemmatize text
    """lmtzr=WordNetLemmatizer()
    text=text.split()
    text=[lmtzr.lemmatize(word) for word in text]
    text=" ".join(text)"""
    
    return text

In [20]:
#function that splits, preprocesses and saves the output
def clean_split(text):
    #text=text.split()
    clean_texts=[]
    split_sent=[]
    
    sent=split_into_sentences(text)
    for j in range(len(sent)):
        processed=clean_text(sent[j])
        if processed!=" ":
            split_sent.append(sent[j])
            clean_texts.append(processed)
    split_output='split_sentences.csv'
    preprocessed_output='processed_sentences.csv'
    
    """file_name=os.path.splitext(os.path.splitext(a_file)[0])[0]
    #print (file_name)
    
    split_output=os.path.join(split_output_dir,file_name+"_split.csv")
    preprocessed_output=os.path.join(preprocessed_output_dir,file_name+"_pre.csv")
    """
    
    with open(preprocessed_output,'w+') as csv_file:
        for line in clean_texts:
            csv_file.write(line)
            csv_file.write('\n')
            
    with open(split_output,'w+') as csv_file:
        for line in split_sent:
            csv_file.write(line)
            csv_file.write('\n')

In [22]:
clean_split(text_pro)